# Solve Sokoban

In [1]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
# print(sys.version)
import numpy as np
from robotic import ry
import time

[rai] util.cpp:initCmdLine:559(1) ** cmd line arguments: 'rai-pybind -python '
[rai] util.cpp:initCmdLine:562(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
[rai] util.cpp:initCmdLine:563(1) ** rai path: '/root/local/rai'
[rai] graph.cpp:initParameters:1364(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}



## Setting up a basic Config

In [2]:
C = ry.Config()
C.clear()

run = 1
file = 23
C.addFile(f'/home/basti/Dokumente/Bachelorarbeit/gym-pcgrl/shared_runs/sokoban_turtle_{run}_log/generated/{file}.g')

C.view()

0

In [3]:
C.getFrameNames()

['world',
 'floor',
 'roboJoint',
 'robo',
 'base',
 'wheel1',
 'wheel2',
 'wheel3',
 'wheel4',
 'bodyRotation',
 'body',
 'head',
 'armL',
 'handL',
 'armR',
 'handR',
 'block0-0',
 'block0-1',
 'block0-2',
 'block0-3',
 'block0-4',
 'block0-5',
 'block0-6',
 'block0-7',
 'block0-8',
 'block0-9',
 'block0-10',
 'block0-11',
 'block1-0',
 'block1-11',
 'block2-0',
 'block2-6',
 'block2-7',
 'block2-11',
 'block3-0',
 'goal3-3',
 'block3-11',
 'block4-0',
 'block4-2',
 'block4-11',
 'block5-0',
 'block5-4',
 'box5-5',
 'goal5-8',
 'block5-11',
 'block6-0',
 'block6-11',
 'block7-0',
 'box7-2',
 'block7-4',
 'block7-11',
 'block8-0',
 'block8-11',
 'block9-0',
 'block9-11',
 'block10-0',
 'block10-4',
 'block10-5',
 'block10-10',
 'block10-11',
 'block11-0',
 'block11-1',
 'block11-2',
 'block11-3',
 'block11-4',
 'block11-5',
 'block11-6',
 'block11-7',
 'block11-8',
 'block11-9',
 'block11-10',
 'block11-11']

## Create skeleton

In [4]:
S = ry.Skeleton()
# touch box3-3
# S.addEntry([1.], ry.SY.touch, ['handL', 'box3-8'])
S.addEntry([1.], ry.SY.positionEq, ['robo', 'goal3-3'])

# # move box4-4 to goal 5-5
# S.addEntry([1.], ry.SY.touch, ['armL', 'box4-4'])
# S.addEntry([1., 2.], ry.SY.stable, ['armL', 'box4-4'])
# S.addEntry([2.], ry.SY.above, ['box4-4', 'goal5-5'])
# S.addEntry([2.,7.], ry.SY.stableOn, ['goal5-5', 'box4-4'])

# # move box2-4 to goal1-1
# S.addEntry([3.], ry.SY.touch, ['armL', 'box2-4'])
# S.addEntry([3., 4.], ry.SY.stable, ['armL', 'box2-4'])
# S.addEntry([4.], ry.SY.above, ['box2-4', 'goal1-1'])
# S.addEntry([4.,7.], ry.SY.stableOn, ['goal1-1', 'box2-4'])

# # move box2-2 to goal5-1
# S.addEntry([5.], ry.SY.touch, ['armL', 'box2-2'])
# S.addEntry([5., 6.], ry.SY.stable, ['armL', 'box2-2'])
# S.addEntry([6.], ry.SY.above, ['box2-2', 'goal5-1'])
# S.addEntry([6.,7.], ry.SY.stableOn, ['goal5-1', 'box2-2'])


## solve for waypoints: create a komo instance, create nlp instance, then call generic solver

In [5]:
komo = S.getKomo_waypoints(C, 1e-1, 1e-2)
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
waypoints = komo.getPath_qAll()
# report on result, view, and play
print(ret)
#print(nlpW.report(2))
komo.view(True, 'waypoints solution')
komo.view_play(True, .2)
# store result

SolverReturn: { time: 0.00081, evals: 27, done: 0, feasible: 1, sos: 0.0609122, f: 0, ineq: 0, eq: 0.170216 }
***** optConstrained: method=AugmentedLagrangian bounds: no
** optConstr. it=0start evals:0 mu=1 nu=1 muLB=0.1 	lambda=[]
** optConstr. it=0start evals:26 f(x)=0.0608389 	g_compl=0 	h_compl=0.451248 	|x-x'|=5 	x=[-2.49749, -2.499, -0.000930031]
** optConstr. it=1end   evals:26 mu=5 nu=5 muLB=0.02 	lambda=[0, 0, 0, 0, 0, 0, 0.502549, 0.199947, -0.2]
** optConstr. it=1start evals:27 f(x)=0.0609122 	g_compl=0 	h_compl=0.170216 	|x-x'|=0.00301505 	x=[-2.5005, -2.5002, -0.000926142]
** optConstr. StoppingCriterion Delta<0.01


1

## solve for paths using RRT: for each phase create start-end problems, run RRT

In [6]:
m = len(waypoints)
rrt_dofs = []
rrt_paths = []
for t in range(0,int(m)):
    # grab config and waypoints
    [Ctmp, q0, q1] = S.getTwoWaypointProblem(t, komo)
    Ctmp.setJointState(q0);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' START')
    Ctmp.setJointState(q1);
    Ctmp.view(True, 'waypoint configuration phase ' + str(t) + ' STOP')

    # call path finder
    sol = ry.PathFinder()
    sol.setProblem(Ctmp, q0, q1)
    ret = sol.solve()
    rrt_paths.append(ret.x)
    rrt_dofs.append(Ctmp.getDofIDs())

    #display the rrt path
    for i in range(0,ret.x.shape[0]):
        Ctmp.setJointState(ret.x[i])
        Ctmp.view(False, 'rrt path ' + str(i))
        time.sleep(.1)

## solve for full path: create a komo instance, initialize with waypoints & rrt paths, solve

In [ ]:
komo = S.getKomo_path(C, 20, .2, -1, 1e-2)
komo.initWithWaypoints(waypoints)
komo.view(True, 'init with waypoints only')
for t in range(0,int(m)):
    komo.initPhaseWithDofsPath(t, rrt_dofs[t], rrt_paths[t], True)
    komo.view(True, 'init with RRT phase ' + str(t))
nlp = komo.nlp()
sol = ry.NLP_Solver()
sol.setProblem(nlp)
sol.setOptions( stopTolerance=1e-2 )
ret = sol.solve()
# report on result, view, and play
print(ret)
#print(nlp.report(2))
komo.view(True, 'path solution')
komo.view_play(True, .2)

In [ ]:
C.view()